In [9]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# URL der CSV-Datei auf GitHub
url = 'https://raw.githubusercontent.com/SilasBereit/turbomachinery-pumpenanalyse/main/volume_flow_data.csv'

# Lade die CSV-Datei mit Semikolon als Trennzeichen und ohne unnötige Spalten
df = pd.read_csv(url, sep=';', usecols=[0, 1])  # Wir verwenden nur die ersten zwei Spalten

# Manuelle Benennung der Spalten
df.columns = ['timestamp', 'flow_rate_m3h']

# Überprüfe die ersten paar Zeilen
print(df.head())

# Datentypen umwandeln
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%d.%m.%y %H:%M')

# m³/h → m³/s
df['flow_rate_m3s'] = df['flow_rate_m3h'] / 3600

# Zeitdifferenz berechnen
df['time_delta_s'] = df['timestamp'].diff().dt.total_seconds()
df['time_delta_s'] = df['time_delta_s'].fillna(0)  # Keine Warnung mehr

# Kennlinie für Impeller Ø264 mm (geschätzt aus Diagramm)
flow_m3h = np.array([110, 240, 300, 350, 425, 495, 555])
head_m = np.array([23, 22, 21, 20, 18, 16, 14.3])

head_interpolator = interp1d(flow_m3h, head_m, bounds_error=False, fill_value="extrapolate")
df['head_m'] = head_interpolator(df['flow_rate_m3h'])

# Konstanten
rho = 1000       # Dichte [kg/m³]
g = 9.81         # Erdbeschleunigung [m/s²]

# Hydraulische Leistung
df['P_hydraulic'] = rho * g * df['flow_rate_m3s'] * df['head_m']  # [Watt]

# Energie
df['E_hydraulic'] = df['P_hydraulic'] * df['time_delta_s']

# Gesamtenergie in kWh
total_energy_joule = df['E_hydraulic'].sum()
total_energy_kWh = total_energy_joule / 3.6e6

print(f"Gesamte hydraulische Energie: {total_energy_kWh:.2f} kWh")


        timestamp  flow_rate_m3h
0  01.04.24 00:00         312.54
1  01.04.24 00:01         193.78
2  01.04.24 00:02         393.96
3  01.04.24 00:03         364.70
4  01.04.24 00:04         100.33
Gesamte hydraulische Energie: 59.75 kWh
